In [25]:
#importing necessary libaries under their alias
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
# read the data output template and assigning it to the variable output. We parse the columns SYEAR AND SMITH as datetime objects and assignd to a new colume name DATE
output= pd.read_excel('data/Data_Output_Template.xlsx', parse_dates={'DATE':['SYEAR','SMTH']})
print(output.nunique()) # print the number o unique values
print(output.shape) # print the shape the number of rows and colums
output.info()   # print the summary of information dataset such as datatypes, column names etc
output

DATE                     264
LMO_Detailed_Industry     59
Employment                 0
dtype: int64
(15576, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15576 entries, 0 to 15575
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   DATE                   15576 non-null  datetime64[ns]
 1   LMO_Detailed_Industry  15576 non-null  object        
 2   Employment             0 non-null      float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 365.2+ KB


,DATE,LMO_Detailed_Industry,Employment
0,1997-01-01,Accommodation services,NaN
1,1997-01-01,Air transportation,NaN
2,1997-01-01,Ambulatory health care services,NaN
3,1997-01-01,"Amusement, gambling and recreation industries",NaN
4,1997-01-01,"Architectural, engineering and related services",NaN
...,...,...,...
15571,2018-12-01,Utilities,NaN
15572,2018-12-01,Warehousing and storage,NaN
15573,2018-12-01,Water transportation,NaN
15574,2018-12-01,Wholesale trade,NaN


In [27]:
# read the LMO Detailed Industries by NAICS file
lmo= pd.read_excel('data/LMO_Detailed_Industries_by_NAICS.xlsx')
print(lmo.shape)
print(lmo.head())
print(lmo.info())

(59, 2)
                             LMO_Detailed_Industry      NAICS
0                                            Farms  111 & 112
1                    Fishing, hunting and trapping        114
2                             Forestry and logging        113
3  Support activities for agriculture and forestry        115
4                           Oil and gas extraction        211
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   LMO_Detailed_Industry  59 non-null     object
 1   NAICS                  59 non-null     object
dtypes: object(2)
memory usage: 1.0+ KB
None


In [28]:
# Cleaning the lmo dataset
lmo['NAICS']=lmo['NAICS'].astype('str').str.replace('&',',').str.split(',') # replaces & with comma and splits the row into a list
lmo= lmo.explode('NAICS') # Transform each element of a list-like to a row, replicating index values
lmo['NAICS']=lmo.NAICS.str.strip() # remove leading or trailing whitespaces in the NAICS values and make them uniform.
lmo.info() 
lmo.tail(10) # Display last 10 rows

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107 entries, 0 to 58
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   LMO_Detailed_Industry  107 non-null    object
 1   NAICS                  107 non-null    object
dtypes: object(2)
memory usage: 2.5+ KB


,LMO_Detailed_Industry,NAICS
51,"Amusement, gambling and recreation industries",713
52,Heritage institutions,712
53,Accommodation services,721
54,Food services and drinking places,722
55,"Repair, personal and non-profit services",81
56,Federal government public administration,911
57,Provincial and territorial public administration,912
58,Local and Indigenous public administration,913
58,Local and Indigenous public administration,914
58,Local and Indigenous public administration,919


In [29]:
data2_files=['data/RTRA_Employ_2NAICS_97_99.csv',
             'data/RTRA_Employ_2NAICS_00_05.csv',
             'data/RTRA_Employ_2NAICS_06_10.csv',
             'data/RTRA_Employ_2NAICS_11_15.csv',
             'data/RTRA_Employ_2NAICS_16_20.csv']

data3_files=['data/RTRA_Employ_3NAICS_97_99.csv',
             'data/RTRA_Employ_3NAICS_00_05.csv',
             'data/RTRA_Employ_3NAICS_06_10.csv'
             ,
             'data/RTRA_Employ_3NAICS_11_15.csv',
             'data/RTRA_Employ_3NAICS_16_20.csv']

data4_files=['data/RTRA_Employ_4NAICS_97_99.csv',
             'data/RTRA_Employ_4NAICS_00_05.csv',
             'data/RTRA_Employ_4NAICS_06_10.csv',
             'data/RTRA_Employ_4NAICS_11_15.csv',
             'data/RTRA_Employ_4NAICS_16_20.csv']

def merge_csv_files(files):
    """  Loads and merges the list of csv files into a single dataframe, parses the year and month column 
        as a new datetime object column and subsets dates from 1997 to 2018
         Args:
              list of local CSV file names.
          Returns:
          A merged dataframe
    """
    a=[pd.read_csv(file, parse_dates={'DATE':['SYEAR','SMTH']}) for file in files ] # a for loop list comprehension to read the datasets
    df=pd.concat(a, ignore_index =True) # concatenate the loaded files
    df= df[(df['DATE'] <'2019')] # subsets dates less than the year 2019
    return df

In [30]:
# apply the merged_csv_files on the 2 digit industries files
data_2=merge_csv_files(data2_files)
print(data_2.shape)
print(data_2.head())
print(data_2.describe())
print(data_2.info())

(5244, 3)
        DATE                                              NAICS  _EMPLOYMENT_
0 1997-01-01               Accommodation and food services [72]        129250
1 1997-01-01  Administrative and support, waste management a...         58500
2 1997-01-01    Agriculture, forestry, fishing and hunting [11]         62250
3 1997-01-01            Arts, entertainment and recreation [71]         38750
4 1997-01-01                                  Construction [23]        118000
        _EMPLOYMENT_
count    5244.000000
mean   108044.574752
std     73417.408347
min         0.000000
25%     52750.000000
50%     93750.000000
75%    161500.000000
max    329000.000000
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5244 entries, 0 to 5243
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   DATE          5244 non-null   datetime64[ns]
 1   NAICS         5244 non-null   object        
 2   _EMPLOYMENT_  5244 non

In [31]:
# Separate industry description from industry code
df1 = pd.DataFrame(data_2['NAICS'].str.split('[').tolist(), columns=["NAICS_name", "CODE"]) # subset the  digit dataset NAICS column and split into a list with two new  columns
df2 =pd.DataFrame(df1["CODE"].astype(str).str.replace(']', '', regex= True).str.split('-')) 
data_2['NAICS']= df2
data_2

,DATE,NAICS,_EMPLOYMENT_
0,1997-01-01,[72],129250
1,1997-01-01,[56],58500
2,1997-01-01,[11],62250
3,1997-01-01,[71],38750
4,1997-01-01,[23],118000
...,...,...,...
5239,2018-12-01,[53],51750
5240,2018-12-01,"[44, 45]",282500
5241,2018-12-01,"[48, 49]",136250
5242,2018-12-01,[22],12250


In [32]:
# Using the pandas explode method to transform each element of a list-like to a row while replicating their index values
data_2=data_2.explode('NAICS')
print(data_2.shape)
data_2.info()

(6036, 3)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6036 entries, 0 to 5243
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   DATE          6036 non-null   datetime64[ns]
 1   NAICS         6036 non-null   object        
 2   _EMPLOYMENT_  6036 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 188.6+ KB


In [33]:
# merge and assign the result to a new varible
dat2=lmo.merge(data_2,on='NAICS', how='right').reset_index(drop= True)
print(dat2)
dat2.info()

                              LMO_Detailed_Industry NAICS       DATE  \
0                                               NaN    72 1997-01-01   
1     Business, building and other support services    56 1997-01-01   
2                                               NaN    11 1997-01-01   
3                                               NaN    71 1997-01-01   
4                                      Construction    23 1997-01-01   
...                                             ...   ...        ...   
6031                                            NaN    45 2018-12-01   
6032                                            NaN    48 2018-12-01   
6033                                            NaN    49 2018-12-01   
6034                                      Utilities    22 2018-12-01   
6035                                Wholesale trade    41 2018-12-01   

      _EMPLOYMENT_  
0           129250  
1            58500  
2            62250  
3            38750  
4           118000  
...      

In [34]:
# apply the merged_csv_files on the 3 digit industries files
data_3= merge_csv_files(data3_files)
print(data_3.shape)
print(data_3.head())
print(data_3.describe()) # prints descriptive statistics about the dataset
print(data_3.info())
data_3

(26928, 3)
        DATE                                     NAICS  _EMPLOYMENT_
0 1997-01-01     Aboriginal public administration[914]           500
1 1997-01-01               Accommodation services[721]         24000
2 1997-01-01  Administrative and support services[561]         56750
3 1997-01-01                   Air transportation[481]         17000
4 1997-01-01      Ambulatory health care services[621]         40000
        _EMPLOYMENT_
count   26928.000000
mean    21040.766117
std     29739.079898
min         0.000000
25%      4750.000000
50%     11000.000000
75%     25000.000000
max    219000.000000
<class 'pandas.core.frame.DataFrame'>
Int64Index: 26928 entries, 0 to 26927
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   DATE          26928 non-null  datetime64[ns]
 1   NAICS         26928 non-null  object        
 2   _EMPLOYMENT_  26928 non-null  int64         
dtypes: datetime64[ns](1), i

,DATE,NAICS,_EMPLOYMENT_
0,1997-01-01,Aboriginal public administration[914],500
1,1997-01-01,Accommodation services[721],24000
2,1997-01-01,Administrative and support services[561],56750
3,1997-01-01,Air transportation[481],17000
4,1997-01-01,Ambulatory health care services[621],40000
...,...,...,...
26923,2018-12-01,Utilities[221],12000
26924,2018-12-01,Warehousing and storage[493],8750
26925,2018-12-01,Waste management and remediation services[562],9000
26926,2018-12-01,Water transportation[483],5000


In [35]:
#Separating the industry name from industry code
df3 = pd.DataFrame(data_3['NAICS'].str.split('[').tolist(), columns=["NAICS_name", "CODE"])
df3 =pd.DataFrame(df3["CODE"].astype('str').str.replace(']','', regex=True))
data_3['NAICS']= df3
data_3['NAICS']= data_3.NAICS.str.strip()
data_3.info()
data_3.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26928 entries, 0 to 26927
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   DATE          26928 non-null  datetime64[ns]
 1   NAICS         26928 non-null  object        
 2   _EMPLOYMENT_  26928 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 841.5+ KB


,DATE,NAICS,_EMPLOYMENT_
0,1997-01-01,914,500
1,1997-01-01,721,24000
2,1997-01-01,561,56750
3,1997-01-01,481,17000
4,1997-01-01,621,40000


In [36]:
# merge and assign the result to a new varible
dat3=lmo.merge(data_3, on='NAICS', how='right').reset_index(drop=True)
print(dat3.nunique())
dat3.info()
dat3

LMO_Detailed_Industry     43
NAICS                    102
DATE                     264
_EMPLOYMENT_             746
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27192 entries, 0 to 27191
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   LMO_Detailed_Industry  19836 non-null  object        
 1   NAICS                  27192 non-null  object        
 2   DATE                   27192 non-null  datetime64[ns]
 3   _EMPLOYMENT_           27192 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 849.9+ KB


,LMO_Detailed_Industry,NAICS,DATE,_EMPLOYMENT_
0,Local and Indigenous public administration,914,1997-01-01,500
1,Accommodation services,721,1997-01-01,24000
2,NaN,561,1997-01-01,56750
3,Air transportation,481,1997-01-01,17000
4,Ambulatory health care services,621,1997-01-01,40000
...,...,...,...,...
27187,NaN,221,2018-12-01,12000
27188,Warehousing and storage,493,2018-12-01,8750
27189,NaN,562,2018-12-01,9000
27190,Water transportation,483,2018-12-01,5000


In [37]:
# apply the merged_csv_files on the 4 digit industries files
data_4=merge_csv_files(data4_files)
print(data_4.shape)
print(data_4.head())
print(data_4.describe())
data_4.info()

(81876, 3)
        DATE  NAICS  _EMPLOYMENT_
0 1997-01-01   1100          3500
1 1997-01-01   1111           250
2 1997-01-01   1112           500
3 1997-01-01   1113          2250
4 1997-01-01   1114          5250
              NAICS   _EMPLOYMENT_
count  81876.000000   81876.000000
mean    4442.924813    6920.046778
std     1765.730560   12738.720881
min     1100.000000       0.000000
25%     3272.000000     750.000000
50%     4431.000000    2500.000000
75%     5323.000000    7500.000000
max     9191.000000  159000.000000
<class 'pandas.core.frame.DataFrame'>
Int64Index: 81876 entries, 0 to 81875
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   DATE          81876 non-null  datetime64[ns]
 1   NAICS         81876 non-null  int64         
 2   _EMPLOYMENT_  81876 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 2.5 MB


In [38]:
data_4['NAICS'] = data_4['NAICS'].astype('str') # change the data type of NAICS column to a string in order to merge on lmo datasets.
dat4=lmo.merge(data_4,on='NAICS',how='right').reset_index(drop=True)
print(dat4.nunique())
dat4.info()

LMO_Detailed_Industry     10
NAICS                    314
DATE                     264
_EMPLOYMENT_             545
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81876 entries, 0 to 81875
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   LMO_Detailed_Industry  6072 non-null   object        
 1   NAICS                  81876 non-null  object        
 2   DATE                   81876 non-null  datetime64[ns]
 3   _EMPLOYMENT_           81876 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 2.5+ MB


In [39]:
# merge all three industy datasets by concatenating them.
combined_industries=pd.concat([dat2,dat3,dat4], ignore_index= True)
combined_industries.info()
combined_industries.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115104 entries, 0 to 115103
Data columns (total 4 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   LMO_Detailed_Industry  27720 non-null   object        
 1   NAICS                  115104 non-null  object        
 2   DATE                   115104 non-null  datetime64[ns]
 3   _EMPLOYMENT_           115104 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 3.5+ MB


LMO_Detailed_Industry      59
NAICS                     439
DATE                      264
_EMPLOYMENT_             1122
dtype: int64

In [40]:
combined_industries.dropna(inplace=True) # drop all null rows without reassignment
combined_industries.drop_duplicates(subset=['DATE','LMO_Detailed_Industry'], inplace = True, ignore_index=True) # drop duplicates of DATE and LMO details columns
print(combined_industries.nunique())
combined_industries.info()
combined_industries.head()

LMO_Detailed_Industry     59
NAICS                     59
DATE                     264
_EMPLOYMENT_             719
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15576 entries, 0 to 15575
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   LMO_Detailed_Industry  15576 non-null  object        
 1   NAICS                  15576 non-null  object        
 2   DATE                   15576 non-null  datetime64[ns]
 3   _EMPLOYMENT_           15576 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 486.9+ KB


,LMO_Detailed_Industry,NAICS,DATE,_EMPLOYMENT_
0,"Business, building and other support services",56,1997-01-01,58500
1,Construction,23,1997-01-01,118000
2,"Repair, personal and non-profit services",81,1997-01-01,89250
3,Real estate rental and leasing,53,1997-01-01,50750
4,Utilities,22,1997-01-01,9750


In [41]:
industry_summary= combined_industries[['DATE','LMO_Detailed_Industry','_EMPLOYMENT_']].sort_values(by=['DATE','LMO_Detailed_Industry'])
industry_summary

,DATE,LMO_Detailed_Industry,_EMPLOYMENT_
1585,1997-01-01,Accommodation services,24000
1586,1997-01-01,Air transportation,17000
1587,1997-01-01,Ambulatory health care services,40000
1588,1997-01-01,"Amusement, gambling and recreation industries",20000
12939,1997-01-01,"Architectural, engineering and related services",27000
...,...,...,...
1582,2018-12-01,Utilities,12250
12933,2018-12-01,Warehousing and storage,8750
12934,2018-12-01,Water transportation,5000
1583,2018-12-01,Wholesale trade,91500


In [42]:
output_summaries= pd.merge(output,industry_summary, on=['DATE','LMO_Detailed_Industry'], how='left')
output_summaries

,DATE,LMO_Detailed_Industry,Employment,_EMPLOYMENT_
0,1997-01-01,Accommodation services,NaN,24000
1,1997-01-01,Air transportation,NaN,17000
2,1997-01-01,Ambulatory health care services,NaN,40000
3,1997-01-01,"Amusement, gambling and recreation industries",NaN,20000
4,1997-01-01,"Architectural, engineering and related services",NaN,27000
...,...,...,...,...
15571,2018-12-01,Utilities,NaN,12250
15572,2018-12-01,Warehousing and storage,NaN,8750
15573,2018-12-01,Water transportation,NaN,5000
15574,2018-12-01,Wholesale trade,NaN,91500


In [43]:
output_summaries['Employment']=output_summaries['_EMPLOYMENT_'] #assign the output employment column to the industry summaries employment 
output_summaries.drop('_EMPLOYMENT_', axis=1,inplace=True) # drop the _EMPLOYMENT column 
output_summaries.set_index('DATE', inplace= True)
output_summaries

,LMO_Detailed_Industry,Employment
DATE,,
1997-01-01,Accommodation services,24000
1997-01-01,Air transportation,17000
1997-01-01,Ambulatory health care services,40000
1997-01-01,"Amusement, gambling and recreation industries",20000
1997-01-01,"Architectural, engineering and related services",27000
...,...,...
2018-12-01,Utilities,12250
2018-12-01,Warehousing and storage,8750
2018-12-01,Water transportation,5000


In [44]:
output_summaries.to_csv('true.csv')